<a href="https://colab.research.google.com/github/Anshuman-37/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IDEA- 
0. Downloading Data and Reading it


> Use KAGGLE JSON file to directly donwload data in colab and use it if having any problem search on google how to download kaggle data.

> ~~Then read the data in pyspark dataframe~~

Done


1. Change Data Type 

> ~~Everything is a string type first convert it to the required type and change the schema~~ 

~~<em>Answer -  I think I will brute force on this one getting all the column names and changing the schema</em>~~

Done

2. Data visualization


> ~~We need to visualize the data and check for imbalance. To do that we need to concatinate the data~~ 

~~Answer - Concatinate Biden and Trumps data in a single data frame~~ 

Done

3. Data cleaning

> Haven't thought about it in detail but we need to do normal nltk steps to do this 

4. Analysis

> Think of using a library or we need to do it on our own ? 

> Do we need to use vector assembler and the other shit as well to do this. I am not sure..












## Downloading the data from Kaggle

In [1]:
## Prerequisites
#!pip install kaggle --upgrade
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
## Downloading data command
!kaggle datasets download -d manchunhui/us-election-2020-tweets

us-election-2020-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!mkdir content/data
!unzip us-election-2020-tweets.zip -d data

mkdir: cannot create directory ‘content/data’: No such file or directory
Archive:  us-election-2020-tweets.zip
replace data/hashtag_donaldtrump.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/hashtag_joebiden.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## Download Pyspark

In [4]:
!pip install pyspark 

## Import Header Files

In [5]:
## Python 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import re


# stopwords, tokenizer, stemmer
import nltk  
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist

# spell correction, lemmatization
from textblob import TextBlob
from textblob import Word

## Spark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType , FloatType
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import udf

# Create Spark Session
spark = SparkSession.builder.master("local[1]").appName('Twitter Sentiment Analysis').getOrCreate()

## Reading the Data

#### Defining the Schema

In [6]:

# Created the schema that we need 
# Fucking Brute Force I dont like it
mySchema = StructType([ StructField('created_at', StringType(), True)\
                       ,StructField('tweet_id', FloatType(), True)\
                       ,StructField('tweet',StringType(),True)\
                       ,StructField('likes',FloatType(),True)\
                       ,StructField('retweet_count',FloatType(),True)\
                       ,StructField('source',StringType(),True)\
                       ,StructField('user_id',FloatType(),True)\
                       ,StructField('user_name',StringType(),True)\
                       ,StructField('user_screen_name',StringType(),True)\
                       ,StructField('user_description',StringType(),True)\
                       ,StructField('user_join_date',StringType(),True)\
                       ,StructField('user_followers_count',FloatType(),True)\
                       ,StructField('user_location',StringType(),True)\
                       ,StructField('lat',FloatType(),True)\
                       ,StructField('long',FloatType(),True)\
                       ,StructField('city',StringType(),True)\
                       ,StructField('country',StringType(),True)\
                       ,StructField('continent',StringType(),True)\
                       ,StructField('state',StringType(),True)\
                       ,StructField('state_code',StringType(),True)\
                       ,StructField('collected_at',StringType(),True)\
                       ])

In [7]:
# Creating spark data frame
# This shit was so hard to do it was not reading multiline data at first it killed me
# Took me 6 hours to figure this shit 
# Now have to change the schema to different data type

trump_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option('inferSchema', "true")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_donaldtrump.csv')
trump_df.cache()
print('Trump DataFrame Schema')
trump_df.printSchema()


biden_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option('inferSchema', "true")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_joebiden.csv')

biden_df.cache()
print('Biden DataFrame Schema')
biden_df.printSchema()

Trump DataFrame Schema
root
 |-- created_at: string (nullable = true)
 |-- tweet_id: float (nullable = true)
 |-- tweet: string (nullable = true)
 |-- likes: float (nullable = true)
 |-- retweet_count: float (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: float (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: float (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- long: float (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)

Biden DataFrame Schema
root
 |-- created_at: string (nullable = true)
 |-- tweet_id: float (nullable = true)
 

#### Trump Data Stats

In [8]:
# Took me 35 seconds to run
trump_df.show(5)

+-------------------+-------------+--------------------+-----+-------------+------------------+------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+----------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|            source|     user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|      city|             country|    continent|               state|state_code|        collected_at|
+-------------------+-------------+--------------------+-----+-------------+------------------+------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+----------+--------------------+-------------+-

In [9]:
# Took me 2 minutes to run one command carefull while running
trump_df.summary().show()

+-------+-----------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|       created_at|            tweet_id|               tweet|            likes|       retweet_count|              source|             user_id|           user_name|    user_screen_name|    user_description|      user_join_date|user_followers_count|user_location|                 lat|                long|                city|             country|           continent|               state|          state_code|        collected_at|
+-------+-----------------+--------------------+--------------------+-----------------+--------------------+----------------

#### Biden Data Stats

In [10]:
biden_df.show(5)

+-------------------+-------------+--------------------+-----+-------------+------------------+-------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+----+--------------------+-------------+----------+----------+--------------------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|            source|      user_id|         user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|      long|city|             country|    continent|     state|state_code|        collected_at|
+-------------------+-------------+--------------------+-----+-------------+------------------+-------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+----+--------------------+-------------+----------+----------+--------------------+
|

In [11]:
# Took me 1 minute 21 seconds to run this command carefull while running
biden_df.summary().show()

+-------+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|       created_at|            tweet_id|               tweet|             likes|       retweet_count|              source|             user_id|           user_name|    user_screen_name|    user_description|      user_join_date|user_followers_count|user_location|                 lat|                long|                city|             country|           continent|               state|          state_code|        collected_at|
+-------+-----------------+--------------------+--------------------+------------------+--------------------+-------------

#### Concatinating Data 

In [12]:
## Adding a column to the dataframe 
trump_df = trump_df.withColumn("Candidate", lit('TRUMP'))
trump_df.show()

+--------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-----------------+--------------------+-------------+--------------------+----------+--------------------+---------+
|          created_at|     tweet_id|               tweet|likes|retweet_count|             source|      user_id|           user_name|   user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|             city|             country|    continent|               state|state_code|        collected_at|Candidate|
+--------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+---------+-------

In [13]:
biden_df = biden_df.withColumn("Candidate", lit('BIDEN'))
biden_df.show()

+--------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+---------+----------+-----------+--------------------+--------------------+--------------------+----------+--------------------+---------+
|          created_at|     tweet_id|               tweet|likes|retweet_count|             source|      user_id|           user_name|   user_screen_name|    user_description|      user_join_date|user_followers_count|       user_location|      lat|      long|       city|             country|           continent|               state|state_code|        collected_at|Candidate|
+--------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+---------+----

In [37]:
data = trump_df.union(biden_df)
data.show()

+--------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-----------------+--------------------+-------------+--------------------+----------+--------------------+---------+
|          created_at|     tweet_id|               tweet|likes|retweet_count|             source|      user_id|           user_name|   user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|             city|             country|    continent|               state|state_code|        collected_at|Candidate|
+--------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+---------+-------

## EDA



```
someone please plot graphs for each one of them
```

In [15]:
## Checking the data imbalance
data.groupBy('Candidate').count().withColumnRenamed("count","Number of Data").show()

+---------+--------------+
|Candidate|Number of Data|
+---------+--------------+
|    TRUMP|       1041276|
|    BIDEN|        823970|
+---------+--------------+



In [16]:
## Comparrsion for the number of tweets
data.groupby('Candidate').agg(F.expr('count(tweet)')\
                              .alias('Number of Tweets')).show()

+---------+----------------+
|Candidate|Number of Tweets|
+---------+----------------+
|    TRUMP|         1008345|
|    BIDEN|          801828|
+---------+----------------+



In [17]:
# comparison of likes
data.groupby('Candidate').agg(F.expr('count(likes)')\
                              .alias('Number of Likes')).show()


+---------+---------------+
|Candidate|Number of Likes|
+---------+---------------+
|    TRUMP|         930039|
|    BIDEN|         749924|
+---------+---------------+



In [18]:
# Can someone please remove the null that is being shown also show only top 10 rows 
data.groupby('Country').count()\
.withColumnRenamed('count','Number of Data')\
.sort(F.desc('Number of Data')).show()


+--------------------+--------------+
|             Country|Number of Data|
+--------------------+--------------+
|                null|       1081590|
|United States of ...|        316181|
|       United States|         59192|
|      United Kingdom|         55419|
|               India|         38786|
|             Germany|         32820|
|              France|         32245|
|              Canada|         26411|
|               Italy|         18524|
|           Australia|         14201|
|              Mexico|         10284|
|              Turkey|          9829|
|     The Netherlands|          9147|
|            Pakistan|          8352|
|              Brazil|          8287|
|               Spain|          6556|
|             Ireland|          5259|
|         Netherlands|          5136|
|            Colombia|          4229|
|           Venezuela|          4041|
+--------------------+--------------+
only showing top 20 rows



In [19]:
# Making USA to US 
data = data.withColumn('Country', regexp_replace('Country', 'United States of America', 'US'))
data = data.withColumn('Country', regexp_replace('Country', 'United States', 'US'))

## Make sure by checking the count of the data 
data.groupby('Country').count()\
.withColumnRenamed('count','Number of Data')\
.sort(F.desc('Number of Data')).show()

+---------------+--------------+
|        Country|Number of Data|
+---------------+--------------+
|           null|       1081590|
|             US|        375374|
| United Kingdom|         55419|
|          India|         38786|
|        Germany|         32820|
|         France|         32245|
|         Canada|         26411|
|          Italy|         18524|
|      Australia|         14201|
|         Mexico|         10284|
|         Turkey|          9829|
|The Netherlands|          9147|
|       Pakistan|          8352|
|         Brazil|          8287|
|          Spain|          6556|
|        Ireland|          5259|
|    Netherlands|          5136|
|       Colombia|          4229|
|      Venezuela|          4041|
|      Argentina|          4022|
+---------------+--------------+
only showing top 20 rows



## Preprocessing the Data(Tweets)

In [20]:
!pip install nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
##### TRY this one by your self
# Refer to user defined functions for this application
# text = u' This dog \U0001f602 <a href="https://stackoverflow.com/questions/49840965/how-to-add-hyperlinks-to-strings-in-python">TIC123</a> '
# print(text) 

# def preprocess_tweets(text):#, stem=False, lemmatize=False):
#     # Make everything in lower case
#     text = text.lower()
#     # Remove links, special characters , numbers punctuation etc. 
#     # Refer - https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python 
#     # emoji_pattern = re.compile("["
#     #     u"\U0001F600-\U0001F64F"  # emoticons
#     #     u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#     #     u"\U0001F680-\U0001F6FF"  # transport & map symbols
#     #     u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#     #                        "]+", flags=re.UNICODE)
#     # Bottom one is also working try it with the example that I am using 
#     # I will comment this out so you guys can try
#     chars_and_links= r'\d+|http?\S+|[^A-Za-z0-9]+'
#     text = re.sub(chars_and_links,' ',text)
#     #text = re.sub(emoji_pattern,' ',text)
#     return text

# print(preprocess_tweets(text)

In [26]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_tweets(text, stem=False, lemmatize=False):
    ## If statement for fault tolerance
    if isinstance(text,str):
        text = text.lower()
        # Remove links, special characters , numbers punctuation etc. 
        chars_and_links= r'\d+|http?\S+|[^A-Za-z0-9]+'
        # Cleaned the text
        text = re.sub(chars_and_links,' ',text)

        filtered_text = []
        # Tokenize the text
        words = word_tokenize(text) 
        # Remove stopwords and stem
        for word in words:
            # Words in stop words or not
            if not word in stop_words:
                if stem:
                    filtered_text.append(ps.stem(word))
                elif lemmatize:
                    filtered_text.append(Word(word).lemmatize())
                else:
                    filtered_text.append(word)
        # Return the filtered text
        return filtered_text
    else:
        return text

In [43]:
# Drop all the NAN values
#data.na.drop(subset=["tweet"]).show()


## Now to apply this to the text we need to create a user defined function
## Refer https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/

## Created my udf
Preprocess_UDF = udf(lambda x: preprocess_tweets(x)) 
data #"marketplace","star_rating").distinct().show(5)
#data = data.select(Preprocess_UDF(F.col("tweet")).alias("tweet")).show(5)
data = data.select([F.col for col in data.columns],Preprocess_UDF(F.col('tweet').alias('Tokenized_tweet'))).show()

ConnectionRefusedError: ignored

In [ ]:
data.show(5)